In [1]:
import pandas as pd
from actionrules.actionRulesDiscovery import ActionRulesDiscovery

In [81]:
emp_data = {'EmployeeID' : [1, 2, 3, 4, 5, 6],
        'Position' : ['Developer', 'Developer', 'Developer', 'Developer', 'Developer','Developer'],
        'Salary' : ['Medium', 'Medium', 'Medium', 'High', 'High', 'High'],
        'Stayed' : ['Yes', 'No', 'No', 'Yes', 'Yes', 'No']}
data = pd.DataFrame(data=emp_data).set_index('EmployeeID')
data

,Position,Salary,Stayed
EmployeeID,,,
1,Developer,Medium,Yes
2,Developer,Medium,No
3,Developer,Medium,No
4,Developer,High,Yes
5,Developer,High,Yes
6,Developer,High,No


In [82]:
util_data = {0 : ['Salary_medium', 'Salary_high', 'Stayed_yes', 'Stayed_no'], 1 : [500, 1000, 700, 0]}
utility = pd.DataFrame(data=util_data).set_index(0)
utility

,1
0,
Salary_medium,500
Salary_high,1000
Stayed_yes,700
Stayed_no,0


In [79]:
actionRulesDiscovery = ActionRulesDiscovery()
actionRulesDiscovery.load_pandas(data)
actionRulesDiscovery.fit(stable_attributes=['Position'],
                         flexible_attributes=['Salary'],
                         consequent='Stayed',
                         conf=40,
                         supp=3,
                         desired_classes=['Yes'],
                         utility_source=utility,
                         min_profit=-300
                         )

In [80]:
actionRulesDiscovery.get_pretty_action_rules()

["If attribute 'Position' is 'developer', attribute 'Salary' value 'medium' is changed to 'high', then 'Stayed' value 'No' is changed to 'Yes' with support: 0.3333333333333333, confidence: 0.4444444444444444 and uplift: 0.16666666666666663. Profit of the action is -266.66666666666674."]